In [7]:
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [8]:
import pandas as pd

In [2]:
torch.cuda.is_available()

True

## Load ROCStories

In [9]:
# Tokenize the input data
train_path = '/h/brandon/internship/Uncover_implicit_bias/gpt2_training/data/ROCStories_winter2017.csv'  # This file should contain one story per line
val_path = 'path_to_validation_data.txt'  # This file should contain one story per line

# Create a dataset and collator
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128
)

/h/brandon/.local/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [21]:
# Function to decode the token IDs
def decode_sample(token_ids):
    return tokenizer.decode(token_ids, clean_up_tokenization_spaces=True, skip_special_tokens=True)

# Visualize the first few samples from train_dataset
print("Some samples from the training set:\n")
for i in range(5):  # let's print out the first 5 samples
    token_ids = train_dataset[i]  # get the token IDs for the i-th sample
    text = decode_sample(token_ids)  # decode the token IDs to text
    print(f"Sample {i + 1}:\n{text}\n")

Some samples from the training set:

Sample 1:
storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight recently.,He examined his habits to try and figure out the reason.,He realized he'd been eating too much fast food lately.,He stopped going to burger places and started a vegetarian diet.,"After a few weeks, he started to feel much better."
0beabab2-fb49-

Sample 2:
460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about his actions.
87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual event.,All of his clothes were either too formal or too casual.,He decided to buy a pair of khakis.,

Sample 3:
The pair he bought fit him perfectly.,Marcus was hap

## Train

In [ ]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',         # Output directory
    overwrite_output_dir=True,      # Overwrite the content of the output directory
    num_train_epochs=3,             # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    per_device_eval_batch_size=4,   # Batch size for evaluation
    eval_steps=400,                 # Evaluation step
    save_steps=800,                 # After # steps model is saved
    warmup_steps=500,               # Warmup steps
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("fine_tuned_gpt2_rocstories")